In [29]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycalphad import Database, Model, variables as v, equilibrium

In [30]:
TDB_PATH = r"NIST-solder.tdb"   # <-- set to your path
OUTDIR = "validation_outputs"

SN_WT_STEPS = 401
X_SN_WT = np.linspace(0.0, 100.0, SN_WT_STEPS)
T_MIN_C, T_MAX_C = 0.0, 400.0
T_STEP_C = 1.0
TS_C = np.arange(T_MIN_C, T_MAX_C + T_STEP_C, T_STEP_C)
TS_K = TS_C + 273.15

# Increase pdens if you see sparse NaNs
CALC_OPTS = {"pdens": 300}

# Phases expected for Pb–Sn; extras harmless if absent
PHASES_OF_INTEREST = [
    "LIQUID",
    "FCC_A1",      # Pb(s)
    "BCT_A5",      # Sn(s)
    "HCP_A3",
    "RHOMBO_A7",
]

ATOMIC_WT = {"PB": 207.2, "SN": 118.71}


def wt_to_mole_frac_sn(wt_sn):
    wt_pb = 100.0 - wt_sn
    n_pb = wt_pb / ATOMIC_WT["PB"]
    n_sn = wt_sn / ATOMIC_WT["SN"]
    return n_sn / (n_sn + n_pb)


def pretty(name):
    mapping = {"LIQUID": "Liq", "FCC_A1": "Pb(s)", "BCT_A5": "Sn(s)", "HCP_A3": "HCP", "RHOMBO_A7": "A7"}
    s = mapping.get(str(name), str(name))
    return s.split(":")[0][:12]


def get_dim_names_for_TXP(eq):
    """Return (T_dim, X_dim, P_dim) for eq['NP'] robustly."""
    NP = eq["NP"]
    dims = list(NP.dims)
    T_dim = "T" if "T" in dims else next(d for d in dims if d.upper().startswith("T"))
    X_candidates = [d for d in dims if d.upper().startswith("X")]
    if not X_candidates:
        raise RuntimeError(f"Could not find composition dim among {dims}")
    X_dim = X_candidates[0]
    if "phase" in NP.coords:
        P_dim = "phase"
    elif "vertex" in dims:
        P_dim = "vertex"
    else:
        P_dim = dims[-1]
    return T_dim, X_dim, P_dim


def phase_names_from(eq, P_dim):
    """Best-effort list of phase names corresponding to the phase axis."""
    NP = eq["NP"]
    if "Phase" in eq:
        return [str(x) for x in np.array(eq["Phase"].values, dtype=object).tolist()]
    if P_dim in NP.coords:
        return [str(x) for x in NP.coords[P_dim].values.tolist()]
    return [f"{P_dim}_{i}" for i in range(NP.sizes[P_dim])]


def align_phase_names_to_axis(phase_names, NP_aligned, P_dim):
    """Ensure phase_names length matches actual last-axis size."""
    nP = NP_aligned.sizes[P_dim]
    if len(phase_names) == nP:
        return phase_names, nP
    if P_dim in NP_aligned.coords:
        return [str(x) for x in NP_aligned.coords[P_dim].values.tolist()], nP
    # pad/truncate as needed
    out = []
    for i in range(nP):
        out.append(phase_names[i] if i < len(phase_names) else f"{P_dim}_{i}")
    return out, nP


def find_phase_index_by_alias(phase_names, target_aliases=("LIQUID", "L", "LIQUIDL", "LIQ")):
    """Robustly find the Liquid phase index given display-name variations."""
    norm = [re.sub(r"[^A-Z0-9_]", "", str(p).upper()) for p in phase_names]
    aliases = [re.sub(r"[^A-Z0-9_]", "", a.upper()) for a in target_aliases]
    for i, n in enumerate(norm):
        if n in aliases or any(a in n for a in aliases):
            return i
    return None


# -----------------------
# PLOTS
# -----------------------
def plot_binary_predominant_map(db):
    phases_present = [p for p in PHASES_OF_INTEREST if p in db.phases]
    if not phases_present:
        raise RuntimeError("None of the requested phases are present in the TDB!")

    x_sn_mole = np.array([wt_to_mole_frac_sn(w) for w in X_SN_WT])
    cond = {v.T: TS_K, v.X("SN"): x_sn_mole}
    comps = ["PB", "SN"]

    print(f"Computing equilibrium on a grid: {len(TS_K)} (T points) × {len(x_sn_mole)} (X points)")
    eq = equilibrium(db, comps, phases_present, cond, calc_opts=CALC_OPTS)

    NP = eq["NP"]
    T_dim, X_dim, P_dim = get_dim_names_for_TXP(eq)
    phase_names = phase_names_from(eq, P_dim)

    # Align to (T, X, *other, P)
    other_dims = [d for d in NP.dims if d not in (T_dim, X_dim, P_dim)]
    NP_aligned = NP.transpose(T_dim, X_dim, *other_dims, P_dim)
    phase_names, nP = align_phase_names_to_axis(phase_names, NP_aligned, P_dim)
    fr = np.asarray(NP_aligned.values)  # (nT, nX, *nOther, nP)

    # Reduce nuisance dims (e.g. 'N')
    if len(other_dims) > 0:
        reduce_axes = tuple(range(2, 2 + len(other_dims)))
        with np.errstate(invalid="ignore"):
            fr = np.nanmax(fr, axis=reduce_axes)  # -> (nT, nX, nP)

    # Predominant phase per (T,X)
    with np.errstate(invalid="ignore"):
        all_nan = np.all(~np.isfinite(fr), axis=-1)
        fr_filled = np.where(np.isfinite(fr), fr, -np.inf)
        imax = np.nanargmax(fr_filled, axis=-1)  # (nT, nX)
        imax[all_nan] = -1

    valid_mask = (imax >= 0) & (imax < nP)
    valid_vals = np.unique(imax[valid_mask])
    unique_raw = sorted([phase_names[int(k)] for k in valid_vals])

    if len(unique_raw) == 0:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.text(0.5, 0.5, "No stable phases found\nin this T–X window",
                ha="center", va="center", transform=ax.transAxes)
        ax.set_axis_off()
        os.makedirs(OUTDIR, exist_ok=True)
        fig.savefig(os.path.join(OUTDIR, "pbsn_predominant_map.png"),
                    dpi=800, bbox_inches="tight")
        plt.close(fig)
        return None

    phase_to_int = {ph: k for k, ph in enumerate(unique_raw)}
    int_grid = np.full(imax.shape, np.nan, dtype=float)
    for i in range(imax.shape[0]):          # over T
        for j in range(imax.shape[1]):      # over X
            k = int(imax[i, j])
            if 0 <= k < nP:
                ph = phase_names[k]
                if ph in phase_to_int:
                    int_grid[i, j] = phase_to_int[ph]

    fig, ax = plt.subplots(figsize=(8, 7.5))
    im = ax.imshow(
        int_grid, origin="lower", aspect="auto",
        extent=[X_SN_WT.min(), X_SN_WT.max(), TS_C.min(), TS_C.max()],
        interpolation="none", resample=False
    )
    ax.set_xlabel("Sn (wt%)")
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("Pb–Sn (binary) — Predominant phase map")

    clean_ticks = [pretty(p) for p in unique_raw]
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_ticks(list(range(len(unique_raw))))
    cbar.set_ticklabels(clean_ticks)
    cbar.set_label("Predominant Phase", rotation=270, labelpad=15)
    cbar.ax.tick_params(labelsize=9)
    for lbl in cbar.ax.get_yticklabels():
        lbl.set_rotation(0); lbl.set_ha("left"); lbl.set_x(1.3)

    plt.tight_layout()
    os.makedirs(OUTDIR, exist_ok=True)
    fig.savefig(os.path.join(OUTDIR, "pbsn_predominant_map.png"), dpi=1000, bbox_inches="tight")
    plt.close(fig)
    return None


def plot_isothermal_slice(db, T_iso_C=183.0):
    phases_present = [p for p in PHASES_OF_INTEREST if p in db.phases]
    if not phases_present:
        raise RuntimeError("None of the requested phases are present in the TDB!")

    x_sn_mole = np.array([wt_to_mole_frac_sn(w) for w in X_SN_WT])
    cond = {v.T: T_iso_C + 273.15, v.X("SN"): x_sn_mole}
    comps = ["PB", "SN"]
    eq = equilibrium(db, comps, phases_present, cond, calc_opts=CALC_OPTS)

    NP = eq["NP"]
    dims = list(NP.dims)
    P_dim = "phase" if "phase" in NP.coords else ("vertex" if "vertex" in dims else dims[-1])
    X_dim = next(d for d in dims if d.upper().startswith("X"))
    phase_names = phase_names_from(eq, P_dim)

    other_dims = [d for d in NP.dims if d not in (X_dim, P_dim)]
    NP_aligned = NP.transpose(X_dim, *other_dims, P_dim)
    phase_names, nP = align_phase_names_to_axis(phase_names, NP_aligned, P_dim)
    fr = np.asarray(NP_aligned.values)  # (nX, *nOther, nP)

    if len(other_dims) > 0:
        reduce_axes = tuple(range(1, 1 + len(other_dims)))
        with np.errstate(invalid="ignore"):
            fr = np.nanmax(fr, axis=reduce_axes)  # -> (nX, nP)

    with np.errstate(invalid="ignore"):
        all_nan = np.all(~np.isfinite(fr), axis=1)
        fr_filled = np.where(np.isfinite(fr), fr, -np.inf)
        imax = np.nanargmax(fr_filled, axis=1)
        imax[all_nan] = -1

    labels = [phase_names[int(k)] for k in imax if (k >= 0 and k < nP)]
    unique = sorted(set(labels))
    if len(unique) == 0:
        return None
    lab_to_int = {lab: i for i, lab in enumerate(unique)}
    y = np.array([lab_to_int[phase_names[int(k)]] if (k >= 0 and k < nP) else np.nan for k in imax])

    fig, ax = plt.subplots(figsize=(8, 3))
    ax.plot(X_SN_WT, y, lw=1.0)
    ax.set_yticks(list(range(len(unique))))
    ax.set_yticklabels([pretty(u) for u in unique])
    ax.set_xlabel("Sn (wt%)")
    ax.set_ylabel("Predominant phase")
    ax.set_title(f"Pb–Sn at {T_iso_C:.1f} °C (isothermal slice)")
    ax.grid(True, alpha=0.3)
    os.makedirs(OUTDIR, exist_ok=True)
    fig.savefig(os.path.join(OUTDIR, f"pbsn_isothermal_{int(round(T_iso_C))}C.png"),
                dpi=800, bbox_inches="tight")
    plt.close(fig)
    return None


def plot_binary_liquidus_solidus(db):
    """Extract and plot liquidus & solidus from the equilibrium grid."""
    phases_present = [p for p in PHASES_OF_INTEREST if p in db.phases]

    # Build the same (T, X) grid
    x_sn_mole = np.array([wt_to_mole_frac_sn(w) for w in X_SN_WT])
    cond = {v.T: TS_K, v.X("SN"): x_sn_mole}
    comps = ["PB", "SN"]
    eq = equilibrium(db, comps, phases_present, cond, calc_opts=CALC_OPTS)

    NP = eq["NP"]

    # Identify axes
    dims = list(NP.dims)
    T_dim = "T" if "T" in dims else next(d for d in dims if d.upper().startswith("T"))
    X_dim = next(d for d in dims if d.upper().startswith("X"))
    if "phase" in NP.coords:
        P_dim = "phase"
        phase_names = [str(x) for x in NP.coords[P_dim].values.tolist()]
    elif "Phase" in eq:
        P_dim = NP.dims[-1]
        phase_names = [str(x) for x in np.array(eq["Phase"].values, dtype=object).tolist()]
    else:
        P_dim = NP.dims[-1]
        phase_names = [f"{P_dim}_{i}" for i in range(NP.sizes[P_dim])]

    # Align to (T, X, *other, P) so we know exactly which names go with the last axis
    other_dims = [d for d in NP.dims if d not in (T_dim, X_dim, P_dim)]
    NP_aligned = NP.transpose(T_dim, X_dim, *other_dims, P_dim)
    fr = np.asarray(NP_aligned.values)  # (nT, nX, *nOther, nP)

    # Ensure phase_names length matches the actual last axis length
    nP = NP_aligned.sizes[P_dim]
    if len(phase_names) != nP:
        if P_dim in NP_aligned.coords:
            phase_names = [str(x) for x in NP_aligned.coords[P_dim].values.tolist()]
            nP = len(phase_names)
        else:
            phase_names = [phase_names[i] if i < len(phase_names) else f"{P_dim}_{i}" for i in range(nP)]

    # Collapse any extra dims (e.g., 'N') by max fraction (we just need presence of Liquid)
    if len(other_dims) > 0:
        reduce_axes = tuple(range(2, 2 + len(other_dims)))
        with np.errstate(invalid="ignore"):
            fr = np.nanmax(fr, axis=reduce_axes)  # -> (nT, nX, nP)

    # --- Find Liquid phase index robustly ---
    # 1) alias-based matching
    import re
    def _find_liquid_idx(names):
        norm = [re.sub(r"[^A-Z0-9_]", "", str(p).upper()) for p in names]
        aliases = [re.sub(r"[^A-Z0-9_]", "", a) for a in ("LIQUID","LIQUIDL","LIQ","L")]
        for i, n in enumerate(norm):
            if n in aliases or any(a in n for a in aliases):
                return i
        return None

    kL = _find_liquid_idx(phase_names)

    # 2) fallback: auto-detect the phase that dominates at the top of T-range (likely Liquid)
    if kL is None:
        # average over the top 30°C of the grid (adjustable)
        nT = fr.shape[0]
        if nT > 10:
            top_slice = slice(max(0, nT - 30), nT)  # ~30 samples (since ΔT=1 °C)
        else:
            top_slice = slice(max(0, nT - 5), nT)
        with np.errstate(invalid="ignore"):
            avg_top = np.nanmean(fr[top_slice, :, :], axis=(0, 1))  # (nP,)
        kL = int(np.nanargmax(avg_top))
        # If you want to be super cautious, you could also check that this phase fraction
        # decreases as T decreases at mid compositions, but in Pb–Sn this heuristic works well.

    if kL is None or not (0 <= kL < nP):
        print("Available phase names on NP phase axis:", phase_names)
        raise RuntimeError("Could not identify a LIQUID-like phase.")

    # Liquid fraction over grid
    frL = fr[..., kL]  # (nT, nX)

    # Build liquidus/solidus arrays (°C)
    liquidus = np.full(frL.shape[1], np.nan, dtype=float)
    solidus  = np.full(frL.shape[1], np.nan, dtype=float)

    for j in range(frL.shape[1]):
        col = frL[:, j]                 # over T
        valid = np.isfinite(col)
        if not np.any(valid):
            continue

        # two-phase (L+solid): 0 < frL < 1
        two_phase = valid & (col > 0.0) & (col < 1.0)
        if np.any(two_phase):
            iL = np.max(np.where(two_phase)[0])   # highest T where two-phase holds
            liquidus[j] = TS_C[iL]

        # solidus: highest T where frL == 0
        no_liq = valid & (col == 0.0)
        if np.any(no_liq):
            iS = np.max(np.where(no_liq)[0])
            solidus[j] = TS_C[iS]

    # Eutectic (min of liquidus)
    eut_mask = np.isfinite(liquidus)
    eut_T = np.min(liquidus[eut_mask]) if np.any(eut_mask) else np.nan
    if np.any(eut_mask):
        eut_index = np.where(eut_mask)[0][np.argmin(liquidus[eut_mask])]
        eut_x = X_SN_WT[eut_index]
    else:
        eut_x = np.nan

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(X_SN_WT, liquidus, lw=1.6, label="Liquidus")
    ax.plot(X_SN_WT, solidus,  lw=1.2, label="Solidus")
    if np.isfinite(eut_T) and np.isfinite(eut_x):
        ax.scatter([eut_x], [eut_T], s=25)
        ax.text(eut_x, eut_T + 6, f"Eutectic ≈ {eut_T:.1f} °C @ {eut_x:.1f} wt% Sn",
                ha="center", va="bottom")
    ax.set_xlim(0, 100)
    ax.set_ylim(TS_C.min(), TS_C.max())
    ax.set_xlabel("Sn (wt%)")
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("Pb–Sn — Liquidus and Solidus (from equilibrium grid)")
    ax.legend()
    ax.grid(True, alpha=0.3)

    os.makedirs(OUTDIR, exist_ok=True)
    fig.savefig(os.path.join(OUTDIR, "pbsn_liquidus_solidus.png"), dpi=900, bbox_inches="tight")
    plt.close(fig)
    return None


# -----------------------
# MAIN
# -----------------------
def main():
    os.makedirs(OUTDIR, exist_ok=True)
    db = Database(TDB_PATH)

    phases_present = [p for p in PHASES_OF_INTEREST if p in db.phases]
    print(f"Using {len(phases_present)} valid phases out of {len(PHASES_OF_INTEREST)} requested.")
    if not phases_present:
        print("Available phases:", list(db.phases.keys()))
        raise SystemExit("No usable phases found in this database for Pb–Sn.")

    plot_binary_predominant_map(db)       # quick smoke test
    plot_binary_liquidus_solidus(db)      # boundary curves for textbook comparison
    plot_isothermal_slice(db, 183.0)      # optional: eutectic slice

    print(f"Figures saved in: {OUTDIR}")
    print("Expected: liquidus min ≈ 183 °C near ~62 wt% Sn;")
    print("two solid solutions (Pb-rich FCC_A1 and Sn-rich BCT_A5) with narrow solubilities.")


if __name__ == "__main__":
    main()


Using 5 valid phases out of 5 requested.
Computing equilibrium on a grid: 401 (T points) × 401 (X points)


/var/folders/r_/mmq2gnpx6qnbz26x3ypfrmvh0000gn/T/ipykernel_81042/3697575681.py:120: RuntimeWarning: All-NaN slice encountered
  fr = np.nanmax(fr, axis=reduce_axes)  # -> (nT, nX, nP)
/var/folders/r_/mmq2gnpx6qnbz26x3ypfrmvh0000gn/T/ipykernel_81042/3697575681.py:278: RuntimeWarning: All-NaN slice encountered
  fr = np.nanmax(fr, axis=reduce_axes)  # -> (nT, nX, nP)
/var/folders/r_/mmq2gnpx6qnbz26x3ypfrmvh0000gn/T/ipykernel_81042/3697575681.py:302: RuntimeWarning: Mean of empty slice
  avg_top = np.nanmean(fr[top_slice, :, :], axis=(0, 1))  # (nP,)


Figures saved in: validation_outputs
Expected: liquidus min ≈ 183 °C near ~62 wt% Sn;
two solid solutions (Pb-rich FCC_A1 and Sn-rich BCT_A5) with narrow solubilities.


/var/folders/r_/mmq2gnpx6qnbz26x3ypfrmvh0000gn/T/ipykernel_81042/3697575681.py:204: RuntimeWarning: All-NaN slice encountered
  fr = np.nanmax(fr, axis=reduce_axes)  # -> (nX, nP)


# pbsn_validate.py — robust CALPHAD smoke-test for Pb–Sn (0–100 wt% Sn, 0–400 °C)

import os
import numpy as np
import matplotlib.pyplot as plt
from pycalphad import Database, equilibrium, variables as v

# -----------------------
# USER INPUTS
# -----------------------
TDB_PATH = r"NIST-solder.tdb"  # e.g., r"../tdb/nist_solder.tdb"
OUTDIR = "validation_outputs"

SN_WT_STEPS = 401
X_SN_WT = np.linspace(0.0, 100.0, SN_WT_STEPS)
T_MIN_C, T_MAX_C = 0.0, 400.0
T_STEP_C = 1.0
TS_C = np.arange(T_MIN_C, T_MAX_C + T_STEP_C, T_STEP_C)
TS_K = TS_C + 273.15

CALC_OPTS = {"pdens": 300}

PHASES_OF_INTEREST = [
    "LIQUID",
    "FCC_A1",
    "BCT_A5",
    "HCP_A3",
    "RHOMBO_A7",
]

ATOMIC_WT = {"PB": 207.2, "SN": 118.71}


def wt_to_mole_frac_sn(wt_sn):
    wt_pb = 100.0 - wt_sn
    n_pb = wt_pb / ATOMIC_WT["PB"]
    n_sn = wt_sn / ATOMIC_WT["SN"]
    return n_sn / (n_sn + n_pb)


def pretty(name):
    mapping = {"LIQUID": "Liq", "FCC_A1": "Pb(s)", "BCT_A5": "Sn(s)", "HCP_A3": "HCP", "RHOMBO_A7": "A7"}
    s = mapping.get(str(name), str(name))
    return s.split(":")[0][:12]


def get_dim_names_for_TXP(eq):
    NP = eq["NP"]
    dims = list(NP.dims)
    T_dim = "T" if "T" in dims else next(d for d in dims if d.upper().startswith("T"))
    X_candidates = [d for d in dims if d.upper().startswith("X")]
    if not X_candidates:
        raise RuntimeError(f"Could not find composition dim among {dims}")
    X_dim = X_candidates[0]
    if "phase" in NP.coords:
        P_dim = "phase"
    elif "vertex" in dims:
        P_dim = "vertex"
    else:
        P_dim = dims[-1]
    return T_dim, X_dim, P_dim


def plot_binary_predominant_map(db):
    phases_present = [p for p in PHASES_OF_INTEREST if p in db.phases]
    if not phases_present:
        raise RuntimeError("None of the requested phases are present in the TDB!")

    x_sn_mole = np.array([wt_to_mole_frac_sn(w) for w in X_SN_WT])
    cond = {v.T: TS_K, v.X("SN"): x_sn_mole}
    comps = ["PB", "SN"]

    print(f"Computing equilibrium on a grid: {len(TS_K)} (T points) × {len(x_sn_mole)} (X points)")
    eq = equilibrium(db, comps, phases_present, cond, calc_opts=CALC_OPTS)

    NP = eq["NP"]
    T_dim, X_dim, P_dim = get_dim_names_for_TXP(eq)

    # --- phase names from dataset (fallbacks allowed) ---
    if "Phase" in eq:
        phase_names = [str(x) for x in np.array(eq["Phase"].values, dtype=object).tolist()]
    elif P_dim in eq.coords:
        phase_names = [str(x) for x in eq.coords[P_dim].values.tolist()]
    else:
        phase_names = [f"{P_dim}_{i}" for i in range(NP.sizes[P_dim])]

    # Align so phase axis is last; keep any extra dims (e.g. 'N') to reduce out
    other_dims = [d for d in NP.dims if d not in (T_dim, X_dim, P_dim)]
    NP_aligned = NP.transpose(T_dim, X_dim, *other_dims, P_dim)
    fr = np.asarray(NP_aligned.values)  # (nT, nX, *nOther, nP)

    # --- ensure phase_names length matches the actual phase axis length ---
    nP = NP_aligned.sizes[P_dim]
    if len(phase_names) != nP:
        # If there is a coordinate on NP_aligned use it; else truncate/pad sanely
        if P_dim in NP_aligned.coords:
            phase_names = [str(x) for x in NP_aligned.coords[P_dim].values.tolist()]
            nP = len(phase_names)
        else:
            phase_names = [phase_names[i] if i < len(phase_names) else f"{P_dim}_{i}" for i in range(nP)]

    # Reduce any extra dims before argmax over phase
    if len(other_dims) > 0:
        reduce_axes = tuple(range(2, 2 + len(other_dims)))
        with np.errstate(invalid="ignore"):
            fr = np.nanmax(fr, axis=reduce_axes)  # -> (nT, nX, nP)

    with np.errstate(invalid="ignore"):
        all_nan = np.all(~np.isfinite(fr), axis=-1)
        fr_filled = np.where(np.isfinite(fr), fr, -np.inf)
        imax = np.nanargmax(fr_filled, axis=-1)  # (nT, nX)
        imax[all_nan] = -1

    # Build legend (bounds-checked)
    valid_mask = (imax >= 0) & (imax < nP)
    valid_vals = np.unique(imax[valid_mask])
    unique_raw = sorted([phase_names[int(k)] for k in valid_vals])

    if len(unique_raw) == 0:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.text(0.5, 0.5, "No stable phases found\nin this T–X window",
                ha="center", va="center", transform=ax.transAxes)
        ax.set_axis_off()
        os.makedirs(OUTDIR, exist_ok=True)
        fig.savefig(os.path.join(OUTDIR, "pbsn_predominant_map.png"),
                    dpi=800, bbox_inches="tight")
        plt.close(fig)
        return None

    phase_to_int = {ph: k for k, ph in enumerate(unique_raw)}

    # --- FILL GRID with bounds check (this was your crash site) ---
    int_grid = np.full(imax.shape, np.nan, dtype=float)
    for i in range(imax.shape[0]):          # over T
        for j in range(imax.shape[1]):      # over X
            k = int(imax[i, j])
            if 0 <= k < nP:
                ph = phase_names[k]
                if ph in phase_to_int:
                    int_grid[i, j] = phase_to_int[ph]

    fig, ax = plt.subplots(figsize=(8, 7.5))
    im = ax.imshow(
        int_grid, origin="lower", aspect="auto",
        extent=[X_SN_WT.min(), X_SN_WT.max(), TS_C.min(), TS_C.max()],
        interpolation="none", resample=False
    )
    ax.set_xlabel("Sn (wt%)"); ax.set_ylabel("Temperature (°C)")
    ax.set_title("Pb–Sn (binary) — Predominant phase map")

    clean_ticks = [pretty(p) for p in unique_raw]
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_ticks(list(range(len(unique_raw))))
    cbar.set_ticklabels(clean_ticks)
    cbar.set_label("Predominant Phase", rotation=270, labelpad=15)
    cbar.ax.tick_params(labelsize=9)
    for lbl in cbar.ax.get_yticklabels():
        lbl.set_rotation(0); lbl.set_ha("left"); lbl.set_x(1.3)

    plt.tight_layout()
    os.makedirs(OUTDIR, exist_ok=True)
    fig.savefig(os.path.join(OUTDIR, "pbsn_predominant_map.png"), dpi=1000, bbox_inches="tight")
    plt.close(fig)
    return None


def plot_isothermal_slice(db, T_iso_C=183.0):
    phases_present = [p for p in PHASES_OF_INTEREST if p in db.phases]
    if not phases_present:
        raise RuntimeError("None of the requested phases are present in the TDB!")

    x_sn_mole = np.array([wt_to_mole_frac_sn(w) for w in X_SN_WT])
    cond = {v.T: T_iso_C + 273.15, v.X("SN"): x_sn_mole}
    comps = ["PB", "SN"]

    eq = equilibrium(db, comps, phases_present, cond, calc_opts=CALC_OPTS)

    NP = eq["NP"]
    dims = list(NP.dims)
    if "phase" in NP.coords:
        P_dim = "phase"
    elif "vertex" in dims:
        P_dim = "vertex"
    else:
        P_dim = dims[-1]
    X_candidates = [d for d in dims if d.upper().startswith("X")]
    if not X_candidates:
        raise RuntimeError(f"Could not find composition dim among {dims}")
    X_dim = X_candidates[0]

    if "Phase" in eq:
        phase_names = [str(x) for x in np.array(eq["Phase"].values, dtype=object).tolist()]
    elif P_dim in eq.coords:
        phase_names = [str(x) for x in eq.coords[P_dim].values.tolist()]
    else:
        phase_names = [f"{P_dim}_{i}" for i in range(NP.sizes[P_dim])]

    other_dims = [d for d in NP.dims if d not in (X_dim, P_dim)]
    NP_aligned = NP.transpose(X_dim, *other_dims, P_dim)
    fr = np.asarray(NP_aligned.values)  # (nX, *nOther, nP)

    if len(other_dims) > 0:
        reduce_axes = tuple(range(1, 1 + len(other_dims)))
        with np.errstate(invalid="ignore"):
            fr = np.nanmax(fr, axis=reduce_axes)  # -> (nX, nP)

    with np.errstate(invalid="ignore"):
        all_nan = np.all(~np.isfinite(fr), axis=1)
        fr_filled = np.where(np.isfinite(fr), fr, -np.inf)
        imax = np.nanargmax(fr_filled, axis=1)
        imax[all_nan] = -1

    nP = len(phase_names)
    labels = [phase_names[int(k)] for k in imax if (k >= 0 and k < nP)]
    unique = sorted(set(labels))
    if len(unique) == 0:
        return None
    lab_to_int = {lab: i for i, lab in enumerate(unique)}
    y = np.array([lab_to_int[phase_names[int(k)]] if (k >= 0 and k < nP) else np.nan for k in imax])

    fig, ax = plt.subplots(figsize=(8, 3))
    ax.plot(X_SN_WT, y, lw=1.0)
    ax.set_yticks(list(range(len(unique))))
    ax.set_yticklabels([pretty(u) for u in unique])
    ax.set_xlabel("Sn (wt%)")
    ax.set_ylabel("Predominant phase")
    ax.set_title(f"Pb–Sn at {T_iso_C:.1f} °C (isothermal slice)")
    ax.grid(True, alpha=0.3)
    os.makedirs(OUTDIR, exist_ok=True)
    fig.savefig(os.path.join(OUTDIR, f"pbsn_isothermal_{int(round(T_iso_C))}C.png"), dpi=800, bbox_inches="tight")
    plt.close(fig)
    return None

def plot_binary_liquidus_solidus(db):
    """
    Extract and plot liquidus and solidus from the same (T, X) grid used by the predominant map.
    Writes: 'pbsn_liquidus_solidus.png'
    """
    phases_present = [p for p in PHASES_OF_INTEREST if p in db.phases]
    if "LIQUID" not in phases_present:
        raise RuntimeError("LIQUID phase not available in this database/runtime window.")

    # Compute the same grid (reuse conditions)
    x_sn_mole = np.array([wt_to_mole_frac_sn(w) for w in X_SN_WT])
    cond = {v.T: TS_K, v.X("SN"): x_sn_mole}
    comps = ["PB", "SN"]
    eq = equilibrium(db, comps, phases_present, cond, calc_opts=CALC_OPTS)

    # Find dimensions robustly
    NP = eq["NP"]
    # Identify phase-axis and phase names
    if "phase" in NP.coords:
        P_dim = "phase"
        phase_names = [str(x) for x in NP.coords[P_dim].values.tolist()]
    elif "Phase" in eq:
        P_dim = NP.dims[-1]
        phase_names = [str(x) for x in np.array(eq["Phase"].values, dtype=object).tolist()]
    else:
        P_dim = NP.dims[-1]
        phase_names = [f"{P_dim}_{i}" for i in range(NP.sizes[P_dim])]
    # Identify temperature and composition dims
    T_dim = "T" if "T" in NP.dims else next(d for d in NP.dims if d.upper().startswith("T"))
    X_dim = next(d for d in NP.dims if d.upper().startswith("X"))

    # Move to (T, X, [other...], phase) and collapse any extra dims
    other_dims = [d for d in NP.dims if d not in (T_dim, X_dim, P_dim)]
    NP_aligned = NP.transpose(T_dim, X_dim, *other_dims, P_dim)
    fr = np.asarray(NP_aligned.values)  # (nT, nX, *nOther, nP)
    if len(other_dims) > 0:
        reduce_axes = tuple(range(2, 2 + len(other_dims)))
        with np.errstate(invalid="ignore"):
            fr = np.nanmax(fr, axis=reduce_axes)  # -> (nT, nX, nP)

    # Get the index of the LIQUID phase in the last axis
    try:
        kL = phase_names.index("LIQUID")
    except ValueError:
        raise RuntimeError("LIQUID phase not in computed phase list; cannot extract boundaries.")

    # Liquid fraction over grid
    frL = fr[..., kL]  # shape (nT, nX)

    # Build liquidus/solidus arrays (°C), default to NaN
    liquidus = np.full(frL.shape[1], np.nan, dtype=float)
    solidus  = np.full(frL.shape[1], np.nan, dtype=float)

    # Along each composition column, scan from high T -> low T
    for j in range(frL.shape[1]):
        col = frL[:, j]  # over T
        # valid finite entries
        valid = np.isfinite(col)

        if not np.any(valid):
            continue

        # Liquidus: highest T where 0 < frL < 1  (two-phase L+solid)
        two_phase = valid & (col > 0.0) & (col < 1.0)
        if np.any(two_phase):
            iL = np.max(np.where(two_phase)[0])
            liquidus[j] = TS_C[iL]  # approx, could refine by interpolation

        # Solidus: highest T where frL == 0 (no liquid)
        no_liq = valid & (col == 0.0)
        if np.any(no_liq):
            iS = np.max(np.where(no_liq)[0])
            solidus[j] = TS_C[iS]

    # Eutectic estimate: the minimum of the liquidus curve
    eut_mask = np.isfinite(liquidus)
    eut_T = np.min(liquidus[eut_mask]) if np.any(eut_mask) else np.nan
    eut_j = np.argmin(liquidus[eut_mask]) if np.any(eut_mask) else None
    eut_x = X_SN_WT[np.where(eut_mask)[0][eut_j]] if eut_j is not None else np.nan

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(X_SN_WT, liquidus, lw=1.6, label="Liquidus")
    ax.plot(X_SN_WT, solidus,  lw=1.2, label="Solidus")

    # annotate eutectic if found
    if np.isfinite(eut_T) and np.isfinite(eut_x):
        ax.scatter([eut_x], [eut_T], s=25)
        ax.text(eut_x, eut_T+6, f"Eutectic ≈ {eut_T:.1f} °C @ {eut_x:.1f} wt% Sn",
                ha="center", va="bottom")

    ax.set_xlim(0, 100)
    ax.set_ylim(TS_C.min(), TS_C.max())
    ax.set_xlabel("Sn (wt%)")
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("Pb–Sn — Liquidus and Solidus (from equilibrium grid)")
    ax.legend()
    ax.grid(True, alpha=0.3)

    os.makedirs(OUTDIR, exist_ok=True)
    fig.savefig(os.path.join(OUTDIR, "pbsn_liquidus_solidus.png"), dpi=900, bbox_inches="tight")
    plt.close(fig)


def main():
    os.makedirs(OUTDIR, exist_ok=True)
    db = Database(TDB_PATH)

    phases_present = [p for p in PHASES_OF_INTEREST if p in db.phases]
    print(f"Using {len(phases_present)} valid phases out of {len(PHASES_OF_INTEREST)} requested.")
    if not phases_present:
        print("Available phases:", list(db.phases.keys()))
        raise SystemExit("No usable phases found in this database for Pb–Sn.")

    plot_binary_predominant_map(db)
    plot_isothermal_slice(db, T_iso_C=183.0)
    plot_binary_liquidus_solidus(db)

    print(f"Figures saved in: {OUTDIR}")
    print("Expected features: clear eutectic near ~183 °C and ~38 wt% Sn;")
    print("liquid above that line, Pb(s)/Sn(s) fields below with narrow solid solubilities.")


if __name__ == "__main__":
    main()
